In [1]:
import math
import numpy as np
from numpy import where
import warnings
warnings.filterwarnings('ignore')

# Worksheet 2

## Useful formula

I have written out the folowing formula to help with the work of this worksheet

In [2]:
def entropy(X: np.array) -> float:
    return - (X * where(X != 0, np.log2(X), 0)).sum()

def joint_entropy(D: np.ndarray) -> float:
    return - (D * where(D != 0, np.log2(D), 0)).sum()

def conditional_entropy(D: np.ndarray, Y: np.array) -> float:
    return joint_entropy(D) - entropy(Y)

def mutual_information(D: np.ndarray) -> float:
    return entropy(D.sum(0)) + entropy(D.sum(1)) - joint_entropy(D)

## Q1: marginal and conditional probabilities

Work out the marginal probability distributions and the $x=a$ conditional probability distribution $P(Y\,|\,X=a) for:

| Y \ X | a | b |
| ---- | ---- | ---- |
| 1 | $\frac{1}{3}$ | $\frac{1}{6}$ |
| 2 | $0$ | $\frac{1}{4}$ |
| 3 | $\frac{1}{8}$ | $\frac{1}{8}$ |

In [3]:
# Define the distribution

D = np.array([
    [1/3, 1/6],
    [0, 1/4],
    [1/8, 1/8]
])

# Marginal distributions in the order [1,2,3,'a','b']

np.concatenate((D.sum(1), D.sum(0)))

array([0.5       , 0.25      , 0.25      , 0.45833333, 0.54166667])

In [4]:
# Conditional probability, given X=a

given_X = D[:,0]

given_X / given_X.sum()

array([0.72727273, 0.        , 0.27272727])

## Q2: working out entropy

Each throw has a $\displaystyle\frac{1}{2}$ chance of being heads, constructing a table:

| throws | probability |
| ---- | ---- |
| 1 | $\frac{1}{2}$ |
| 2 | $\frac{1}{4}$ |
| 3 | $\frac{1}{8}$ |
| 4 | $\frac{1}{16}$ |
| 5 | $\frac{1}{32}$ |
| ... | ... |

As you can see from the code below as $\text{throws} \rightarrow \infty$ then $H(X) = 2$

In [5]:
# 2a.

D = np.array([1/2**x for x in range(1, 500)])

entropy(D)

2.0

For a more theoretical answer, the set of possible outcomes is $X = \{1,2,3,...\}$, we need to start by working out $p_{x^n}$.

To get $X=n$ you need to throw $n-1$ tails, for which the probability is $\displaystyle\frac{1}{2^{n-1}}$ followed by a head which has the probability of $\displaystyle\frac{1}{2}$, and therefore $p_x(n)=\displaystyle\frac{1}{2^n}$

To calculate entropy, use the formula:

$H(X) = \displaystyle-\sum_{n\in\mathcal{X}}\frac{1}{2^n}\log{2^{-n}}$

Which cancels to

$\displaystyle\sum_{n=1}^\infty\frac{n}{2^n} = \sum_{n=1}^\infty\frac{n}{\frac{1}{1-2}}=2$

### 2b.

Since you can ask the question "is it $n$?" aka the outcome you are after, this also has a $\frac{1}{2}$ chance of being correct so the average number of questions would also be 2

## Q3: A puzzle which lends itself to information type reasoning

Suppose you have $n$ coins, among which there may or may not be one counterfit coin. If there is a counterfit coin it will either way less or more than the other coins. The coins are weighted using a balance, any number of coins can be put on each side of the balance.

There are $2n+1$ possibilities that the coins can be arranged in

$\{R_1, R_2, R_3, ..., R_N\}$

$\{F_1, R_2, R_3, ..., R_N\}$

$\{R_1, F_2, R_3, ..., R_N\}$

$\{R_1, R_2, F_3, ..., R_N\}$

....

$\{R_1, R_2, R_3, ..., F_N\}$

Since the coins can either be heavier or lighter it is $2\times$

If you work out the entropy

$H(X) = -\displaystyle2n+1\frac{1}{2n+1}\log\frac{1}{2n+1} $

if you simplyify this you get

$H(X) = \log{2n+1}$

For weighing stratagies, $H(Y)$ there are three possible outcomes, it is even, it tips left or right, therefore the probability is $\frac{1}{3}$

$H(Y) = 3\times-\displaystyle\frac{1}{3}\log\frac{1}{3} = \log3$

Since weighing each coin individually (taking $2n+1$ steps) and having an effective weighing stratagy gives you the same information $H(X)= H(Y)$

But our weighing stratagy should take less time (on average) than an exaustive search and therefore

$H(X) \leq k\log3$

where $k$ is the number of weighs performed. Now substitute in $H(X)$ and:

$\displaystyle\log2n+1 \leq k\log3$

$\displaystyle\log2n+1 \leq \log3^k$

$\displaystyle 2n+1 \leq 3^k$

$\displaystyle n \leq \frac{3^k-1}{2}$

### 3b.


Using $k=3$ and $12$ coins, what is the best weighing stratagy

In [31]:
class Coin():
    
    def __init__(self, weight: int = 2):
        self.weight = weight
        
    def __str__(self):
        return str(self.weight)

class Coins():
    
    def __init__(self, index: int, coins: int = 12):
        self.index = index
        self.coins = [Coin() for i in range(1,coins+1)]
        self._initialise_weights()
    
    def _initialise_weights(self):
        self.coins[self.index].weight = 3
        
coins = Coins(2)

## Q4: Working out entropy and information

Let $p(x,y)$ be given by $p(0,0)=p(0,1)=p(1,1)=\displaystyle\frac{1}{3}$ and $p(1,0)=0$:

| Y\X | 0 | 1 |
| ---- | ---- | ---- |
| 0 | $\frac{1}{3}$ | 0 |
| 1 | $\frac{1}{3}$ | $\frac{1}{3}$ |

Find $H(X), H(Y), H(X\,|\,Y), H(Y\,|\,X), H(X, Y), H(Y) - H(Y\,|\,X)$ and $I(X;Y)$

In [7]:
# Define the distribution

P = np.array([
    [1/3, 0],
    [1/3, 1/3]
])

In [8]:
# H(X)

entropy(P.sum(0))

0.9182958340544896

In [9]:
# H(Y)

entropy(P.sum(1))

0.9182958340544896

In [10]:
# H(X|Y)

conditional_entropy(P, P.sum(1))

0.6666666666666665

In [11]:
# H(Y|X)

conditional_entropy(P, P.sum(0))

0.6666666666666665

In [12]:
# H(X,Y)

joint_entropy(P)

1.584962500721156

In [13]:
# H(Y) - H(Y|X)

entropy(P.sum(1)) - conditional_entropy(P, P.sum(0))

0.25162916738782304

## Q5: A qeustion about information in the brain

The original idea of estimating neural information by binning spike trains was spread across several papers, but one of the main references is ... One aspect of this paper we didn't discuss is the use of extrapolation to estimate the information as the number of samples becomes large based on the behaviour for smaller numbers of samples.

Can you give a short, up to five line, summary of what this involves.